In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.block0 = nn.Sequential(
          nn.Conv2d(1, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1),
        )
        self.block1 = nn.Sequential(
          nn.Conv2d(16, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1)
        )
        self.block1d = nn.Sequential(
          nn.Conv2d(16, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1),
        )
        self.block2 = nn.Sequential(
          nn.Conv2d(16, 32, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout(0.1),
        )
        self.block3 = nn.Sequential(
          nn.Conv2d(32, 16, 3, bias=False),
          nn.ReLU(),
          nn.BatchNorm2d(16),
        )
        self.block4 = nn.Sequential(
          nn.Conv2d(16, 10, 1, bias=False),
        )
        self.pool1 = nn.AvgPool2d(2, 2)
        self.pool2 = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.pool1(self.block1(self.block1d(self.block0(x))))
        x = self.block4(self.pool2(self.block3(self.block2(x))))
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,304
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
          Dropout-12           [-1, 16, 22, 22]               0
        AvgPool2d-13           [-1, 16, 11, 11]               0
           Conv2d-14             [-1, 3

<ipython-input-2-3f84d89a3834>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 89888376.53it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 72973309.53it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21634471.16it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14779308.59it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("Epoch number ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch number  1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-3f84d89a3834>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.08337432146072388 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.39it/s]



Test set: Average loss: 0.0756, Accuracy: 9823/10000 (98.23%)

Epoch number  2


loss=0.036148156970739365 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.42it/s]



Test set: Average loss: 0.0482, Accuracy: 9869/10000 (98.69%)

Epoch number  3


...


Test set: Average loss: 0.0208, Accuracy: 9931/10000 (99.31%)

Epoch number  18


loss=0.005579816177487373 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.60it/s]



Test set: Average loss: 0.0174, Accuracy: 9943/10000 (99.43%)

Epoch number  19


loss=0.011081268079578876 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.69it/s]



Test set: Average loss: 0.0172, Accuracy: 9943/10000 (99.43%)

